# SU2 Drudge Example

In this example, we use the SU2 Drudge to show the various components - vectors, terms, etc. in the drudge structure. We then show how to use these objects to do some simple algebra by using the commutation relations.

## Initializing stuff

`pyspark` is a general-purpose parallel cluster-computing framework.
`dummy_spark` is a serialized emulator for the same.

In [1]:
# from pyspark import SparkContext
from dummy_spark import SparkContext
ctx = SparkContext()

In [2]:
from sympy import *
from drudge import *

Initialize the SU2LatticeDrudge. More parameters can be passed into the initialization - check by using the help option

In [3]:
dr = SU2LatticeDrudge(ctx)

In [4]:
help(SU2LatticeDrudge)

Help on class SU2LatticeDrudge in module drudge.su2:

class SU2LatticeDrudge(drudge.genquad.GenQuadLatticeDrudge)
 |  Drudge for a lattice of su(2) algebras.
 |  
 |  This drudge has the commutation rules for :math:`\mathfrak{su}(2)` algebras
 |  in Cartan-Weyl form (Ladder operators).  Here both the shift and Cartan
 |  operators can have additional *lattice indices*.  Operators on different
 |  lattice sites always commute.  In detail, with a generator of the Cartan
 |  subalgebra denoted :math:`h`, and its raising and lowering operators by
 |  :math:`e` and :math:`f`, the commutation rules among the generators with the
 |  same lattice index can be summarized as,
 |  
 |  .. math::
 |  
 |      [h, e] &= root \cdot e \\
 |      [h, f] &= -root \cdot f \\
 |      [e, f] &= norm \cdot h + shift \\
 |  
 |  where :math:`root`, :math:`norm`, and :math:`shift` are all tunable.  This
 |  is basically the common Serre relations.
 |  
 |  Parameters
 |  ----------
 |  
 |  ctx
 |      The S

Every Drudge comes with a `namespace` which stores the names to various attributes/elements of the drudge.

In [5]:
p = dr.names
j_z = p.J_
j_p = p.J_p
j_m = p.J_m

The basic operators are recognized as `Vectors` - these are the building blocks of a `Term`

In [6]:
type(j_z)

drudge.term.Vec

In [7]:
x = (j_z | j_p)
type(x)

drudge.term.Terms

Terms / Vectors alone do not come with display, simplify, etc. options. These are associated with Tensors.

In [8]:
jp = dr.sum(j_p)
jm = dr.simplify(j_m)
jz = dr.einst(j_z)

In [9]:
type(jz)

drudge.drudge.Tensor

In [10]:
jz.simplify().display()

<IPython.core.display.Math object>

In [11]:
jp.display()

<IPython.core.display.Math object>

In [12]:
jm.display()

<IPython.core.display.Math object>

## Commutators

Let's verify the SU(2) commutation rules

\begin{align}
[ J^+ , J^-] &= 2J^z\\
[ J^z , J^+] &= J^+\\
[ J^z , J^-] &= -J^-
\end{align}

**First commutator**

In [13]:
comm1 = (jp | jm)
comm1.display()

<IPython.core.display.Math object>

In [14]:
comm1 = dr.simplify(comm1)
comm1.display()

<IPython.core.display.Math object>

**Second Commutator**

In [15]:
comm2 = (jz | jp)

In [16]:
comm2.simplify().display()

<IPython.core.display.Math object>

## Coupled Cluster (CCD) for a Spin Hamiltonian

We will use the Lipkin model Hamiltonian here to demonstrate CCD equations generation. Hamiltonian is parametrized by interaction strengths `x` and `y`

$$
H = xJ^z + y \left( J^+ J^+ + J^- J^-\right)
$$

In [17]:
#Constructing the Hamiltonian
hx = Symbol('x')
hy = Symbol('y')

ham = hx*jz + hy*(jp*jp + jm*jm)
ham.simplify().display()

<IPython.core.display.Math object>

The Hartree Fock part of the Hamiltonian is

$$ H_0 = x J^z $$

So we use 

$$ T = t J^+J^+ $$

to build Coupled Cluster excitations.

In [18]:
#Cluster excitation operator definition
t = Symbol('t')

clusters = t * jp*jp

Use the Baker-Campbell-Hausdorff expansion to build the effective Hamiltonian $\bar{H}$

In [19]:
#Similarity Transform of the Hamiltonian
%time

curr = ham
h_bar = ham
for order in range(0,5):
    curr = (curr | clusters).simplify() / (order + 1)
    h_bar += curr

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs


Every tensor has an attribute `n_terms` that keeps a track of the total number of terms

In [20]:
h_bar.n_terms

15

In [21]:
h_bar.display()

<IPython.core.display.Math object>

**MERGE function**: Generally, when we simplify any expression in drudge or sympy, the resulting expression is a expanded set of normal-ordered terms. But if you see the above expression, there are several terms which have the same operator part but different coefficients. The `merge` function grabs these operators together.

In [22]:
h_bar.merge().map2amps(factor).display()

<IPython.core.display.Math object>

## Preparing Reports

Jupyter-notebooks are generally quite-convenient to see outputs - but when you are **in a galaxy far far away ...** and prefer using python scripts to do the job instead, the display() command cannot do what it wants to.

In such cases, you can use the `report` functionality of drudge, which can print out the expressions into an HTML or a LaTeX file

In [23]:
with dr.report('report.html','Title of the Report') as rep:
    rep.add('Hamiltonian',ham)
    rep.add('Cluster Operator', clusters)
    rep.add('HBar',h_bar.merge())

with dr.report('report.tex','Title of the Report') as rep:
    rep.add('Hamiltonian',ham)
    rep.add('Cluster Operator', clusters)
    rep.add('HBar',h_bar.merge())

## The Bind and the Filter Functions

Now, we want to compute the expressions for CCD energy and amplitude equations. Unlike fermionic drudges like `GenMBDrudge` or `PartHoleDrudge`, there is no inbuilt `eval_vev` or `eval_fermi_vev` (VEV: Vacuum Expectation Value) function for SU2 drudge.

For Lipkin, while evaluating the expectation value over the Hartree Fock state, only the terms that include all $J^z$ survives (after normal ordering).

Let's say 

\begin{equation}
\langle \phi \left \vert J^z \right \vert \phi \rangle = -N/2
\end{equation}

then we can extract the expectation value by visual inspection or something similar with the help of the `filter` and `bind` functions in drudge.

**Filter out the terms with $J^z$ only**

    Syntax: Tensor.filter( function )
    function returns True / False for the terms we want to keep / discard respectively

In [24]:
h_bar.merge().display()

<IPython.core.display.Math object>

In [25]:
def jz_filter(term):
    vecs = term.vecs
    if len(vecs)==0:
        return True
    elif all(v.base == dr.cartan for v in vecs):
        return True
    else:
        return False

In [26]:
energy_op = h_bar.filter(jz_filter)
energy_op.display()

<IPython.core.display.Math object>

**Replace Vectors / Operators with the scalar expectation value**

    Syntax: Tensor.bind( function )
    function: maps the vectors (or more appropriately terms) into some scalar (or maybe another operator)

First we define a function which creates the map

In [27]:
def get_vev_of_term(term):
    vecs = term.vecs
    m = Symbol('N')
    if len(vecs) == 0:
        return term
    
    if all(i.base == j_z for i in vecs):
        return [Term(sums=term.sums, amp=term.amp * (-m / 2) ** len(vecs), vecs=())]
    else:
        return []


Energy equation

In [28]:
en_eqn = h_bar.bind(get_vev_of_term)
en_eqn.simplify().display()

<IPython.core.display.Math object>

Amplitude equation $ \quad R = \langle \phi \left \vert J^-J^- \bar{H} \right \vert \phi \rangle$

In [29]:
jm2_hbar = (jm * jm * h_bar).simplify()

In [30]:
r_eq = jm2_hbar.bind(get_vev_of_term)

In [31]:
r_eq.simplify().display()

<IPython.core.display.Math object>